In [150]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
from torch.utils.data import DataLoader

In [168]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

full_data = torchvision.datasets.ImageFolder(
                    root='./data/trainset/',
                    transform=transforms
                    )

test_data = torchvision.datasets.ImageFolder(
                    root= './data/test/', 
                    transform=transforms)

size = [int(len(train_data)*0.8), len(train_data) - int(len(train_data)*0.8)]

train_dataset, valid_dataset = torch.utils.data.random_split(dataset=full_data, lengths=size)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )



test_loader = torch.utils.data.DataLoader(
                    test_data,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

In [169]:
class Classifier(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(256, 2)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())

In [170]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [144]:
clf = Classifier()
if cuda_available:
    clf = clf.cuda()
optimizer = torch.optim.SGD(clf.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [175]:
epoches= 50
for epoch in range(epoches):
    losses = []
    # Train
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = clf(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())
        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    clf.eval()
    total = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print('--------------------------------------------------------------')
    clf.train()

Epoch : 0 Loss : 0.693 
Epoch : 0 Loss : 0.693 
Epoch : 0 Loss : 0.693 
Epoch : 0 Loss : 0.693 
Epoch : 0 Loss : 0.693 
Epoch : 0 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 1 Loss : 0.694 
Epoch : 1 Loss : 0.693 
Epoch : 1 Loss : 0.693 
Epoch : 1 Loss : 0.693 
Epoch : 1 Loss : 0.693 
Epoch : 1 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 2 Loss : 0.693 
Epoch : 2 Loss : 0.693 
Epoch : 2 Loss : 0.693 
Epoch : 2 Loss : 0.693 
Epoch : 2 Loss : 0.693 
Epoch : 2 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 3 Loss : 0.693 
Epoch : 3 Loss : 0.693 
Epoch : 3 Loss : 0.693 
Epoch : 3 Loss : 0.693 
Epoch : 3 Loss : 0.693 
Epoch : 3 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 4 Loss : 0.694 
Epoch : 4 Loss : 0.693 
Epoch : 4 Loss : 0.693 
Epoch : 4 Loss : 0.693 
Epoch : 4 Loss : 0.693 
Epoch : 4 Test Acc : 49.000
--------

Epoch : 38 Loss : 0.693 
Epoch : 38 Loss : 0.693 
Epoch : 38 Loss : 0.693 
Epoch : 38 Loss : 0.693 
Epoch : 38 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 39 Loss : 0.693 
Epoch : 39 Loss : 0.693 
Epoch : 39 Loss : 0.693 
Epoch : 39 Loss : 0.693 
Epoch : 39 Loss : 0.693 
Epoch : 39 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 40 Loss : 0.693 
Epoch : 40 Loss : 0.693 
Epoch : 40 Loss : 0.693 
Epoch : 40 Loss : 0.693 
Epoch : 40 Loss : 0.693 
Epoch : 40 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 41 Loss : 0.693 
Epoch : 41 Loss : 0.693 
Epoch : 41 Loss : 0.693 
Epoch : 41 Loss : 0.693 
Epoch : 41 Loss : 0.693 
Epoch : 41 Test Acc : 49.000
--------------------------------------------------------------
Epoch : 42 Loss : 0.694 
Epoch : 42 Loss : 0.693 
Epoch : 42 Loss : 0.693 
Epoch : 42 Loss : 0.693 
Epoch : 42 Loss : 0.693 
Epoch : 42 Test Acc : 49.000
---